
## Peer-graded Assignment: Построение baseline-решений
Настало время перейти к построению моделей! Давайте начнем с построения так называемых бейзлайнов - построим несколько моделей, которые в дальнейшем будем использовать в качестве первого приближения для будущей модели. Часто для решения подобных задач используются линейные модели, а также ансамбли, например, случайный лес или градиентный бустинг.

В этом задании вам предстоит построить несколько моделей и оценить их качество. Эти модели будут служить нам в качестве baseline-решений и пригодятся сразу для нескольких задач:

- Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем большинстве этапов должны использоваться готовые протестированные инструменты. Все это приводит к тому, что baseline-модели - это дешевый способ грубо оценить потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.
- Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе предположить, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.
- Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.
- Наконец, если после построения сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.  

Задание будет оцениваться на основе jupyter notebook'а, который вам нужно будет приложить в качестве решения. Убедитесь, что он содержит всю проделанную вами работу, код написан аккуратно и понятно, его легко читать, а также в тексте notebook'а присутствуют необходимые для понимания проделанной вами работы комментарии.

## Инструкции
Обучите 3 разные baseline-модели на полученных наборах данных и оцените их качество.

На прошлой неделе вы выбрали методику оценки качества моделей на основе кросс-валидации, а также основную и вспомогательные метрики. Оцените с их помощью получившуюся модель.

Обратите внимание, что под разными моделями понимаются именно разные алгоритмы классификации. Например, 2 модели, реализующие метод k ближайших соседей с разными k, будут считаться одним baseline-решением (хотя и с разными параметрами).

Напоминаем, что отложенная выборка (hold-out dataset) не должна использоваться для построения и оценки baseline-моделей!

Можно (но не обязательно) рассмотреть следующий набор алгоритмов:

- Линейная модель (например, реализация sklearn.linear_model.RidgeClassifier)
- Случайный лес (например, реализация sklearn.ensemble.RandomForestClassifier)
- Градиентный бустинг (например, реализация sklearn.ensemble.GradientBoostingClassifier)   

В качестве решения приложите получившийся jupyter notebook. Убедитесь, что в нем присутствуют:

- все baseline-модели, которые вы построили;
- качество всех построенных моделей оценено с помощью кросс-валидации, и это понятно из текста в jupyter notebook;
- все модели оценены с помощью основной и дополнительных метрик качества.

In [214]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")

In [215]:
x_data = pd.read_csv('orange_small_churn_data.train.txt')
y_data = pd.read_csv('orange_small_churn_labels.train.txt', header=None)
y_data.columns = ['Churn']
y_data['Churn'] = [0 if x == -1 else x for x in y_data['Churn']]

In [216]:
Na_columns = []

for column in x_data.columns: # Колонки, полностью состоящие из NaN
    if x_data[column].isna().sum() == x_data.shape[0]:
        x_data.drop(column, axis=1,inplace=True)
        Na_columns.append(column)

x_data.loc[:,'Var191':] = x_data.loc[:,'Var191':].fillna('Na_cat') # Замена NaN категор.признаков соотв.категорией

# Колонки с полностью отсутствующими значениями

In [217]:
X, X_hold_out, y, y_hold_out = train_test_split(x_data, 
                                                    y_data,
                                                    test_size=0.1, shuffle=False, random_state=42) 

In [218]:
for column in X.loc[:, 'Var1':'Var190']:
    median = X[column].dropna()[X[column].dropna() != 0].median()
    X[column].fillna(median, inplace=True)


In [219]:
le = LabelEncoder()
for column in X.loc[:, 'Var191':]:
    X[column] = le.fit_transform(X[column])

In [220]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [221]:
from sklearn.preprocessing import StandardScaler # Использование стандартизации

scaler = StandardScaler()

scaler.fit(X_train.loc[:, 'Var1':'Var190'])

X_train.loc[:, 'Var1':'Var190'] = scaler.transform(X_train.loc[:, 'Var1':'Var190'])
X_test.loc[:, 'Var1':'Var190'] = scaler.transform(X_test.loc[:, 'Var1':'Var190'])

In [222]:
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import (GridSearchCV,
                                     train_test_split,
                                     StratifiedKFold)

In [223]:
skf = StratifiedKFold(n_splits=5, random_state=17)

In [224]:
#Построение моделей

# Logistic Regression

logreg = LogisticRegression(penalty='l2', random_state=42) #l2 при мультиколлинеарности признаков
logreg_model = logreg.fit(X_train, y_train)
lr_predictions = logreg.predict(X_test)
lr_recall = recall_score(y_test, lr_predictions)
lr_f1_score= f1_score(y_test, lr_predictions)
lr_roc_auc = roc_auc_score(y_test, lr_predictions)

In [225]:
print(lr_recall, lr_f1_score, lr_roc_auc)

0.0 0.0 0.5


In [226]:
# Gradient Boosting

gbc = GradientBoostingClassifier(random_state=42)
gbc_model = gbc.fit(X_train, y_train)
gbc_predictions = gbc.predict(X_test)
gbc_recall = recall_score(y_test, gbc_predictions)
gbc_f1_score = f1_score(y_test, gbc_predictions)
gbc_roc_auc = roc_auc_score(y_test, gbc_predictions)

In [227]:
print(gbc_recall, gbc_f1_score, gbc_roc_auc)

0.009140767824497258 0.017825311942959002 0.5038939973197339


In [228]:
# Random Forest

random_forest = RandomForestClassifier(random_state=42)
rfc_model = random_forest.fit(X_train, y_train)
rfc_predictions = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
rfc_recall = recall_score(y_test, rfc_predictions)
rfc_f1_score = f1_score(y_test, rfc_predictions)
rfc_roc_auc = roc_auc_score(y_test, rfc_predictions)

In [229]:
print(rfc_recall, rfc_f1_score, rfc_roc_auc)

0.0 0.0 0.5


In [230]:
gaussian = GaussianNB()
gauss_model = gaussian.fit(X_train, y_train)
gauss_predictions = gaussian.predict(X_test)
gauss_recall = recall_score(y_test, gauss_predictions)
gauss_f1_score= f1_score(y_test, gauss_predictions)
gauss_roc_auc = roc_auc_score(y_test, gauss_predictions)

In [231]:
gauss_recall

0.946983546617916

In [232]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
knn_recall = recall_score(y_test, knn_predictions)
knn_f1_score= f1_score(y_test, knn_predictions)
knn_roc_auc = roc_auc_score(y_test, knn_predictions)

In [233]:
knn_recall

0.003656307129798903